In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import glob
import xml.etree.ElementTree as ET
import fnmatch
import os,sys
import random
from PIL import Image

Read all Annotations and store in bufffer

In [7]:
# Choose train2014 or val2014         
# This is specifically for coco dataset because file name ccontains the information that image is in train set
# or val set and we search this string in file name to load the annotations
#img_set = 'train2014'
img_set = 'val2014'

# Path where all annotations are stored
path = 'Annotations/' + '*.xml' 
#path = 'test_annotations/' + '*.xml' 
files = glob.glob(path)
print len(files)

14536


Crop Images for each class and store it in respective folder and generate labels and store them in labels directory

In [8]:
print 'Generating Data and Labels............'

search_str = '*' + img_set + '*'

# chnage destination directory location to where you wants to store the cropped images
img_dir = '5_class_dataset/images/' + img_set[:-4] + '/'
label_dir = '5_class_dataset/labels/'

# create directories if not exist
dir = os.path.dirname(img_dir)
if not os.path.exists(dir):
    os.makedirs(dir)

# creating directory for labels
dir = os.path.dirname(label_dir)
if not os.path.exists(dir):
    os.makedirs(dir)
        
# label file names
label_filename = label_dir + img_set[:-4] + '.txt'

laptop_state_labels = []
scissor_state_labels = []
suitcase_state_labels = []
toilet_state_labels = []
umbrella_state_labels = []

#Iterating through each annotation file
for infile in files:
    # check if train or val annotations
    if fnmatch.fnmatch(infile, search_str):
        
        tree = ET.parse(infile)
        root = tree.getroot()
        
        # Extract image name
        filename = root.find('filename').text
        file_path = root.find('path').text
        
        # load the image 
        img = Image.open(file_path)
        
        # loop through objects in annotation and extract object name and bbox
        objects = root.findall('object')
        for idx, obj in enumerate(objects):
            name = obj.find('name').text
            
            # extract class and state info... example: suitcase_openSuitcase
            cls, state = name.split('_')
            
            # extract image patch
            bndbox = obj.find('bndbox')
            x1 = bndbox.find('xmin').text
            y1 = bndbox.find('ymin').text
            x2 = bndbox.find('xmax').text
            y2 = bndbox.find('ymax').text
            
            new_img = img.crop((int(x1),int(y1),int(x2),int(y2)))
            
            # creating new file name for cropped image and save cropped into specific folder
            new_name = img_dir + filename + '_' + str(idx) + '.jpg'           
            new_img.save(new_name)
            
            # generating label and appending it to train/ val .txt
            # label 0: laptop, 1:scissor, 2:suitcase, 3:toilet, 4: umbrella
            label = ''  
            with open(label_filename, 'a+') as f:
                if 'laptop' in cls:
                    label = filename + '_' + str(idx) + '.jpg' + ' ' + '0' + '\n'
                    if 'open' in state:
                        st_lbl = filename + '_' + str(idx) + '.jpg' + ' ' + '0' + '\n'
                    else:
                        st_lbl = filename + '_' + str(idx) + '.jpg' + ' ' + '1' + '\n'
                    laptop_state_labels.append(st_lbl)
                    
                elif 'scissor' in cls:
                    label = filename + '_' + str(idx) + '.jpg' + ' ' + '1' + '\n'
                    if 'open' in state:
                        st_lbl = filename + '_' + str(idx) + '.jpg' + ' ' + '0' + '\n'
                    else:
                        st_lbl = filename + '_' + str(idx) + '.jpg' + ' ' + '1' + '\n'
                    scissor_state_labels.append(st_lbl)
                    
                elif 'suitcase' in cls:
                    label = filename + '_' + str(idx) + '.jpg' + ' ' + '2' + '\n'
                    if 'open' in state:
                        st_lbl = filename + '_' + str(idx) + '.jpg' + ' ' + '0' + '\n'
                    else:
                        st_lbl = filename + '_' + str(idx) + '.jpg' + ' ' + '1' + '\n'
                    suitcase_state_labels.append(st_lbl)
                    
                elif 'toilet' in cls:
                    label = filename + '_' + str(idx) + '.jpg' + ' ' + '3' + '\n'
                    if 'open' in state:
                        st_lbl = filename + '_' + str(idx) + '.jpg' + ' ' + '0' + '\n'
                    else:
                        st_lbl = filename + '_' + str(idx) + '.jpg' + ' ' + '1' + '\n'
                    toilet_state_labels.append(st_lbl)
                    
                elif 'umbrella' in cls:
                    label = filename + '_' + str(idx) + '.jpg' + ' ' + '4' + '\n'
                    if 'open' in state:
                        st_lbl = filename + '_' + str(idx) + '.jpg' + ' ' + '0' + '\n'
                    else:
                        st_lbl = filename + '_' + str(idx) + '.jpg' + ' ' + '1' + '\n'
                    umbrella_state_labels.append(st_lbl)
                
                # writting class labels to train/val .txt
                f.write(label)
             
    
print 'Done.'

Generating Data and Labels............
Done.


In [9]:
# Writing state labels to correspoding class file for labelling in t-SNE visual
laptop_st_fname = label_dir + img_set[:-4] + '_laptop_state.txt'
scissor_st_fname = label_dir + img_set[:-4] + '_scissor_state.txt'
suitcase_st_fname = label_dir + img_set[:-4] + '_suitcase_state.txt'
toilet_st_fname = label_dir + img_set[:-4] + '_toilet_state.txt'
umbrella_st_fname = label_dir + img_set[:-4] + '_umbrella_state.txt'

with open(laptop_st_fname, 'w+') as f:
    for val in laptop_state_labels:
        f.write(val)
        
with open(scissor_st_fname, 'w+') as f:
    for val in scissor_state_labels:
        f.write(val)
        
with open(suitcase_st_fname, 'w+') as f:
    for val in suitcase_state_labels:
        f.write(val)
        
with open(toilet_st_fname, 'w+') as f:
    for val in toilet_state_labels:
        f.write(val)
        
with open(umbrella_st_fname, 'w+') as f:
    for val in umbrella_state_labels:
        f.write(val)